# SPCS Networking Connectivity Test: Hubspot

**Note: This Notebook should be run in an SPCS Container for testing to be valid**

1. Check that the Network Rule structure matches your requirements.
2. Run the Connectivity Test if your EAI is already configured.
3. Use the Cells to prepare the EAI if needed, then rerun the test.

In [ ]:
# Hubspot Connectivity Test Configuration

# This role will be used to create the EAI and other objects if necessary
IMPLEMENTATION_ROLE = "ACCOUNTADMIN"

# Hubspot API URL for connectivity test
HUBSPOT_API_TEST_URL = "https://api.hubapi.com/crm/v3/objects/companies/search"

print(f"Configuration:")
print(f"Hubspot API test URL: {HUBSPOT_API_TEST_URL}")
print("\nReady to test connectivity...")

In [ ]:
import requests
from urllib.parse import urlparse

def test_connectivity(url, description):
    """Test HTTP connectivity to a given URL"""
    try:
        print(f"\n🔍 Testing {description}: {url}")

        # Simple HTTP test - let EAI handle the routing
        response = requests.get(url, timeout=10, allow_redirects=True)
        print(f"  ✅ HTTP Response: {response.status_code}")

        # Any response means connectivity is working
        return True

    except requests.exceptions.ConnectionError as e:
        print(f"  ❌ Connection Error: {e}")
        return False
    except requests.exceptions.Timeout as e:
        print(f"  ❌ Timeout Error: {e}")
        return False
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False

# Run connectivity tests
print("=" * 60)
print("HUBSPOT CONNECTIVITY TEST")
print("=" * 60)

results = {}
results['hubspot_api'] = test_connectivity(HUBSPOT_API_TEST_URL, "HUBSPOT API")

print("\n" + "=" * 60)
print("CONNECTIVITY TEST SUMMARY")
print("=" * 60)

all_passed = True
for service, passed in results.items():
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"{service.upper()}: {status}")
    if not passed:
        all_passed = False

if all_passed:
    print(f"\n🎉 All connectivity tests PASSED!")
    print("Your SPCS environment can reach all required Hubspot endpoints.")
    print("You can proceed with Hubspot integration.")
else:
    print(f"\n⚠️ Some connectivity tests FAILED!")
    print("You need to configure Snowflake External Access Integration (EAI).")
    print("Please run the EAI setup cells below to resolve connectivity issues.")

If Connectivity testing fails, you can use the queries below to prepare and implement an EAI suitable for Hubspot API access.

In [ ]:
-- Create Network Rules for Hubspot connectivity
-- Run this cell if connectivity tests failed

-- Network rule for Hubspot API
CREATE OR REPLACE NETWORK RULE hubspot_connectivity_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('api.hubapi.com');

SHOW NETWORK RULES LIKE 'hubspot_%';

In [ ]:
use role {{IMPLEMENTATION_ROLE}};
CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION hubspot_eai
  ALLOWED_NETWORK_RULES = (hubspot_connectivity_rule)
  ENABLED = TRUE
  COMMENT = 'External Access Integration for Hubspot connectivity';

  SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'hubspot_eai';

In [ ]:
ALTER NOTEBOOK EAI_HUBSPOT
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('HUBSPOT_EAI');

After Creating and Setting the EAI on the Notebook, run the Connectivity test again.

**NOTE: Remember to restart your compute session**

In [ ]:
-- Check if network rules exist
SHOW NETWORK RULES LIKE 'hubspot_%';

In [ ]:
-- Check if EAI exists
SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'hubspot_eai';